For the Code in Tensorflow/Keras, please refer: https://www.kaggle.com/code/priyankdl/cifar10-with-transferlearning-flow-from-directory

The following code is written in PyTorch. If you like the notebook, please upvote it.

In [18]:
import numpy as np 
import pandas as pd # for reading csv file

#train and test images are .7z archives, we need to unpack them
!pip install py7zr

from py7zr import unpack_7zarchive
import shutil

#Before using, we need to register unpack format
shutil.register_unpack_format('7zip', ['.7z'], unpack_7zarchive)

#unpack train images in /kaggle/working or /kaggle/temp
shutil.unpack_archive('/kaggle/input/cifar-10/train.7z', '/kaggle/temp/')

In [19]:
import torch
if torch.cuda.is_available():
    device=torch.device(type="cuda", index=0)
else:
    device=torch.device(type="cpu", index=0)            

In [20]:
#first fetching the class names from trainLabels.csv

train_labels=pd.read_csv("/kaggle/input/cifar-10/trainLabels.csv", header='infer')

#unique labels
classes=train_labels['label'].unique()

#confirming
print(classes)

#classnames to classids
name2num={}
i=0
for name in classes:
    name2num[name]=i
    i=i+1

num2name={}
for i in range(len(classes)):
    num2name[i]=classes[i]

['frog' 'truck' 'deer' 'automobile' 'bird' 'horse' 'ship' 'cat' 'dog'
 'airplane']


In [21]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import os
from torchvision.io import read_image
from torchvision.transforms import ToTensor, Normalize, Resize, Compose

class TrainDataset(Dataset):
    def __init__(self, imgpath, labelpath):
        super().__init__()
        self.imgpath=imgpath
        self.labelpath=labelpath
        self.labels=pd.read_csv(labelpath, header='infer')
        self.transform=Compose([Resize((224,224), antialias=True), Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])])
        
    def __len__(self):
        return self.labels.shape[0]
    
    def __getitem__(self,idx):
        finalpath=os.path.join(self.imgpath,str(idx+1))+'.png'
        img=read_image(finalpath)/255
        img=self.transform(img)
        label=self.labels.iloc[idx,1]
        label=name2num[label]
        return img,label

traindataset=TrainDataset('/kaggle/temp/train','/kaggle/input/cifar-10/trainLabels.csv')        
        
batch_size=64    
traindataloader=DataLoader(dataset=traindataset, batch_size=batch_size)

In [22]:
import torch.nn as nn
from torchvision.models import mobilenet_v3_large, MobileNet_V3_Large_Weights
class Cifar10Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.pretrainednet=mobilenet_v3_large(weights=MobileNet_V3_Large_Weights.DEFAULT)
        self.pretrainednet.classifier=nn.Sequential(
            nn.Linear(in_features=960, out_features=1280, 
                   bias=True),nn.Hardswish(), 
            nn.Dropout(p=0.2, inplace=True), 
            nn.Linear(in_features=1280, out_features=10, 
                      bias=True)
        )
        
    def forward(self,x):
        x=self.pretrainednet(x)
        return x

In [23]:
def train_one_epoch(dataloader, model,loss_fn, optimizer):
    model.train()
    track_loss=0
    num_correct=0
    num_param=0
    
    for i, (imgs, labels) in enumerate(dataloader):
        imgs=imgs.to(device)
        labels=labels.to(device)
        pred=model(imgs)
                    
        loss=loss_fn(pred,labels)
        track_loss+=loss.item()
        num_correct+=(torch.argmax(pred,dim=1)==labels).type(torch.float).sum().item()
        
        running_loss=round(track_loss/(i+(imgs.shape[0]/batch_size)),2)
        running_acc=round((num_correct/((i*batch_size+imgs.shape[0])))*100,2)
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if i%100==0:
            print("Batch:", i+1, "/",len(dataloader), "Running Loss:",running_loss, "Running Accuracy:",running_acc)
            
    epoch_loss=running_loss
    epoch_acc=running_acc
    return epoch_loss, epoch_acc

In [24]:
model=Cifar10Net()
model=model.to(device)

for param in model.pretrainednet.features.parameters():
    param.requires_grad=False

loss_fn=nn.CrossEntropyLoss()
lr=0.001
#optimizer=torch.optim.SGD(params=model.parameters(), lr=lr)
optimizer=torch.optim.Adam(params=model.parameters(), lr=lr)
n_epochs=5

for i in range(n_epochs):
    print("Epoch No:",i+1)
    train_epoch_loss, train_epoch_acc=train_one_epoch(traindataloader,model,loss_fn,optimizer)
    print("Training:", "Epoch Loss:", train_epoch_loss, "Epoch Accuracy:", train_epoch_acc)
    print("--------------------------------------------------")

for param in model.pretrainednet.features.parameters():
    param.requires_grad=True

for i in range(n_epochs):
    print("Epoch No:",i+1)
    train_epoch_loss, train_epoch_acc=train_one_epoch(traindataloader,model,loss_fn,optimizer)
    print("Training:", "Epoch Loss:", train_epoch_loss, "Epoch Accuracy:", train_epoch_acc)
    print("--------------------------------------------------")

Epoch No: 1
Batch: 1 / 782 Running Loss: 2.3 Running Accuracy: 15.62
Batch: 101 / 782 Running Loss: 1.06 Running Accuracy: 64.43
Batch: 201 / 782 Running Loss: 0.92 Running Accuracy: 68.23
Batch: 301 / 782 Running Loss: 0.87 Running Accuracy: 69.95
Batch: 401 / 782 Running Loss: 0.83 Running Accuracy: 71.21
Batch: 501 / 782 Running Loss: 0.81 Running Accuracy: 71.78
Batch: 601 / 782 Running Loss: 0.8 Running Accuracy: 72.16
Batch: 701 / 782 Running Loss: 0.79 Running Accuracy: 72.62
Training: Epoch Loss: 0.78 Epoch Accuracy: 72.99
--------------------------------------------------
Epoch No: 2
Batch: 1 / 782 Running Loss: 0.62 Running Accuracy: 75.0
Batch: 101 / 782 Running Loss: 0.67 Running Accuracy: 76.7
Batch: 201 / 782 Running Loss: 0.64 Running Accuracy: 77.11
Batch: 301 / 782 Running Loss: 0.64 Running Accuracy: 77.49
Batch: 401 / 782 Running Loss: 0.64 Running Accuracy: 77.67
Batch: 501 / 782 Running Loss: 0.64 Running Accuracy: 77.64
Batch: 601 / 782 Running Loss: 0.64 Running 

In [26]:
class TestDataset(Dataset):
    def __init__(self, imgpath):
        super().__init__()
        self.imgpath=imgpath
        _,_,self.files=next(os.walk(self.imgpath))
        self.length=len(self.files)
        self.transform=Compose([Resize((224,224), antialias=True), Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])])        
    
    def __len__(self):
        return self.length
    
    def __getitem__(self,idx):
        finalpath=os.path.join(self.imgpath,str(idx+1))+'.png'
        img=read_image(finalpath)/255.0
        img=self.transform(img)
        return img

testdataset=TestDataset('/kaggle/temp/test/')
testdataloader=DataLoader(dataset=testdataset, batch_size=batch_size)

In [27]:
def eval(dataloader, model,loss_fn, path):
    model.eval()
    data=pd.read_csv(path)
    with torch.no_grad():
        for i, imgs in enumerate(dataloader):
            finalbatchpred=np.zeros(imgs.shape[0],dtype='object')
            imgs=imgs.to(device)
            pred=model(imgs)
            
            pred=torch.argmax(pred,dim=1).type(torch.int).cpu()
            for j,p in enumerate(pred):
                finalbatchpred[j]=num2name[p.item()]
            data.iloc[i*batch_size:i*batch_size+batch_size ,1]=finalbatchpred
    
    data.to_csv('submission.csv', index=False)
    data.head()

In [28]:
eval(testdataloader, model,loss_fn, '/kaggle/input/cifar-10/sampleSubmission.csv')

If you liked the notebook, please upvote it.